<a href="https://colab.research.google.com/github/Meluiscruz/C4_TC_Data_Science/blob/AMZ_testing/amazon_scraper_boxes_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Start

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import

In [4]:
#For the system
import os

#Manage of time
from datetime import datetime, timedelta
from pytz import timezone
import time

#Manage of files
import pandas as pd
import csv

#scrap
from bs4 import BeautifulSoup
from openpyxl.workbook import Workbook
import requests

#Extract soup

In [5]:
def extract_soup(url, preview=True):
    response = requests.get(url)
    status = response.status_code

    soup = BeautifulSoup(response.text, 'lxml')

    if preview==True:
        print(soup.prettify())

    return soup, status

In [6]:
    # test_url = 'https://www.amazon.com.mx/gp/bestsellers/musical-instruments/ref=zg_bs_nav_0'
    # test_soup, test_status = extract_soup(test_url, False)
    # print(test_status)
    # print(type(test_status))

#Top amazon boxes

In [7]:
def top_amazon_boxes(soup):
    boxes = soup.find_all('div', attrs={'class':"a-section a-spacing-none aok-relative"})

    return boxes

#Centered String

In [8]:
def centered_string(string):
    rest = 21 - len(string)
    n_spaces = int(rest/2)

    string_spaces = ' ' * n_spaces
    
    centered_string = string_spaces + string + string_spaces

    if len(centered_string)<21:
        centered_string = ' ' + centered_string

    return centered_string

#Update CSV

In [9]:
def update_csv(file, rows, sep=','):
    with open(file, "a", encoding="utf-8") as csv_file:
        csv_file = csv.writer(csv_file, delimiter=sep)
        for row in rows:
            csv_file.writerow(row)

#Scrap boxes

In [19]:
def scrap_boxes(boxes, domain, dict_to_rows=False):
    
    ranks = [None]*50
    product_names = [None]*50
    image_urls = [None]*50
    product_links = [None]*50
    star_ratings = [None]*50
    reviews = [None]*50
    authors_companies = [None]*50
    editions_consoles = [None]*50
    min_prices = [None]*50
    max_prices = [None]*50
    time_log = [None]*50

    amz_mx_url = f'https://www.amazon.com.{domain}'
    
    n_box = 0
    for box in boxes:
        MX__COL_tz = 'America/Mexico_City'
        timezone_MXCOL = timezone(MX__COL_tz) 
        datetime_box = datetime.now(timezone_MXCOL)
        time_log[n_box] = datetime_box.strftime('%Y-%m-%d %H:%M')

        rank_box = box.find_all('span', attrs={'class':'zg-badge-text'})
        products_and_image_box = box.find_all('div', attrs={'class' : 'a-section a-spacing-small'})
        product_links_box = box.find_all('a', attrs={'class' : 'a-link-normal'})
        star_ratings_box = box.find_all('span', attrs={'class' : 'a-icon-alt'})
        reviews_box = box.find_all('a', attrs={'class' : 'a-size-small a-link-normal'})
        authors_company_box = box.find_all('span', attrs={'class' : 'a-size-small a-color-base'})
        editions_console_box = box.find_all('span', attrs={'class' : 'a-size-small a-color-secondary'})
        prices_box = box.find_all('span', attrs={'class' : "p13n-sc-price"})
        
        
        ranks[n_box] = int(rank_box[0].get_text()[1:])

        #In case the element was removed (yes, it happens)
        try:
            product_names[n_box] = products_and_image_box[0].img.get('alt')
            image_urls[n_box] = products_and_image_box[0].img.get('src')
            product_links[n_box] = amz_mx_url + product_links_box[0].get('href')
        except: pass

        try:
            star_ratings[n_box] = float(star_ratings_box[0].get_text()[:3])
            reviews[n_box] = int(reviews_box[0].get_text().replace(',',''))
        except: pass
        #Individual cases
        try:
            authors_companies[n_box] = authors_company_box[0].get_text()
        except:pass
        
        try:
            editions_consoles[n_box] = editions_console_box[0].get_text()
        except:pass

        #Courrencies
        if domain == 'mx':
            coin_symbol = 1
        elif domain == 'br':
            coin_symbol = 2

        try:
            min_prices[n_box] = float(prices_box[0].get_text()[coin_symbol:].replace(',',''))
        except: pass

        try:    
            max_prices[n_box] = float(prices_box[1].get_text()[coin_symbol:].replace(',',''))
        except: pass

        n_box = n_box + 1 

    # Dictionary
    boxes_dict = {
    'time' : time_log,
    "Rank" : ranks,
    "Product Names": product_names,
    "Image urls": image_urls,
    "Product links": product_links,
    "Stars": star_ratings,
    "Reviews": reviews,
    "Authors/Company": authors_companies,
    "Edition/Console": editions_consoles,
    "Price_std_or_min" : min_prices,
    "Max_prices" : max_prices
    }

    if dict_to_rows == True:
        dict_rows = [None]*50
        for n in range(len(dict_rows)):
            dict_rows[n] = [
                boxes_dict["time"][n],
                boxes_dict["Rank"][n],
                boxes_dict["Product Names"][n],
                boxes_dict["Image urls"][n],
                boxes_dict["Product links"][n],
                boxes_dict["Stars"][n],
                boxes_dict["Reviews"][n],
                boxes_dict["Authors/Company"][n],
                boxes_dict["Edition/Console"][n],
                boxes_dict["Price_std_or_min"][n],
                boxes_dict["Max_prices"][n]]

    return boxes_dict, dict_rows

##mx dictionary

In [11]:
mx_dict = {
    'top_food_and_drinks':'grocery',
    'top_automotive':'automotive',
    'top_baby':'baby',
    'top_sports':'sports',
    'top_amazon_devices':'amazon-devices',
    'top_electronics':'electronics',
    'top_tools':'tools',
    'top_kitchen':'kitchen',
    'top_industrial_emp_cience':'industrial',
    'top_musical_instruments':'musical-instruments',
    'top_toys':'toys',
    'top_books':'books',
    'top_music':'music',
    'top_officeproduct':'officeproduct',
    'top_movies':'dvd',
    'top_handmade':'handmade',
    'top_pet_supplies':'pet-supplies',
    'top_fashion':'shoes',
    'top_health':'hpc',
    'top_software':'software',
    'top_kindle':'digital-text',
    'top_videogames':'videogames'
    }

##br ditionary

In [12]:
br_dict = {
    'top_amazon_devices':'amazon-devices',
    'top_baby':'baby-products',
    'top_sports':'sports',
    'top_electronics':'electronics',
    'top_tools':'hi',
    'top_kitchen':'kitchen',
    'top_toys':'toys',
    'top_books':'books',
    'top_music':'music',
    'top_officeproduct':'office',
    'top_movies':'dvd',
    'top_pet_supplies':'pet-products',
    'top_fashion':'fashion',
    'top_health':'hpc',
    'top_software':'computers',
    'top_kindle':'digital-text',
    'top_videogames':'videogames',
    'top_apps':'mobile-apps',
    'top_beauty':'beauty',
    'top_home':'home',
    'top_home_appliances':'appliances',
    'top_pool_garden':'lawn-and-garden',
    'top_furniture':'furniture'
}

##domain dictionary

In [13]:
domain_dict = {'Mexico' : 'mx',
               'Brazil' : 'br'}

##domain path

In [14]:
domain_path = '4SS/4SS_db/testing/{}'

#scrap amazon

In [15]:
def scrap_amazon():
    MX__COL_tz = 'America/Mexico_City'
    timezone_MXCOL = timezone(MX__COL_tz) 

    log_status = [None]*45
    log_date = [None]*45
    log_domain = [None]*45
    log_category = [None]*45

    Start_datetime = datetime.now(timezone_MXCOL)
    Start_datetime = Start_datetime.strftime('%Y-%m-%d %H_%M')
    centered_date = centered_string(Start_datetime)

    n_log = 0

    country_count = 0
    print(f'      Log Date    | Country |        category       | status | Loaded |')

    for country_name in domain_dict:
        country_count = country_count + 1

        if country_count > 1:
            print(' ---------------- | ------- | --------------------- | ------ | ------ |')

        country = domain_dict[country_name]

        if country == 'mx':
            categories_dict = mx_dict

        elif country == 'br':
            categories_dict = br_dict

        for key in categories_dict:
            #Date-time
            log_datetime = datetime.now(timezone_MXCOL)
            log_date[n_log] = log_datetime.strftime('%Y-%m-%d %H:%M')
            print(f' {log_date[n_log]}', end=' | ')
            
            #Country where you are at
            log_domain[n_log] = country
            print(f'   {log_domain[n_log]}  ', end=' | ')

            #Category that is scraping
            category = categories_dict[key]
            log_category[n_log] = category
            centered_category = centered_string(log_category[n_log])
            print(centered_category, end=' | ')
            
            #The main scrap
            url = f'https://www.amazon.com.{country}/gp/bestsellers/{category}/ref=zg_bs_nav_0'
            soup, status = extract_soup(url, preview=False)
            
            if status == 503:
                while status == 503:
                    time.sleep(1)
                    soup, status = extract_soup(url, preview=False)
                    log_status[n_log] = status
                    log_date[n_log] = datetime.now()

            if status ==200:
                log_status[n_log] = status
                print(f'  {log_status[n_log]} ', end=' | ')
                
                boxes = top_amazon_boxes(soup)
                amz_key_top_boxes, amz_rows = scrap_boxes(boxes, country, dict_to_rows=True)
                add_df = pd.DataFrame.from_dict(amz_key_top_boxes)

                #Updating csv files
                csv_file = f'/{country}-master_db_{category}.csv'
                dir_csv_testing = f'/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/Masters/{country}/csv/{csv_file}'
                update_csv(dir_csv_testing, amz_rows, sep='|')
                
                #Updating Parquet files
                parquet_file = f'/{country}-master_db_{category}.parquet'
                dir_parquet_maindb = f'/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/Masters/{country}/parquet/{parquet_file}'
                main_df = pd.read_parquet(dir_parquet_maindb)
                main_df = pd.concat([main_df, add_df])
                #case of NaN values
                main_df['Rank'] = pd.to_numeric(main_df['Rank'], downcast='float')
                #Updating  file
                main_df.to_parquet(dir_parquet_maindb)

                excel_file = f'/{country}-master_db_{category}.xlsx'
                dir_excel_maindb = f'/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/Masters/{country}/excel/{excel_file}'
                #For the testing
                main_df.to_excel(dir_excel_maindb, index=False)

                print('  Yes  |')

            else:
                log_status[n_log] == status
                print(f'  {log_status[n_log]} ', end=' | ')
                
                log_date[n_log] = datetime.now()
                print(' +.No.+ |')

            n_log = n_log + 1

    log_dict = {
        'log_date' : log_date,
        'category' : log_category,
        'country' : log_domain,
        'status' : log_status
    }


    dict_rows= [
        log_date,
        log_category,
        log_domain,
        log_status]

    log_file = f'/master_db_logs.csv'
    dir_log_testing = f'/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/Masters/{log_file}'

    try:
        update_csv(dir_log_testing, log_rows, sep='|')
    except:
        log_df = pd.DataFrame.from_dict(log_dict)
        log_df.to_csv(dir_log_testing, index = False)



    log_print = """
                                ----------------------
                                |   Log file loaded   |
                                ----------------------

    """
    print(log_print)

#Scrap Amazon n times

In [16]:
def scrap_amazon_times(n_times, minutes):
    counter = 0
    for t in range(n_times):
        for min in range(minutes):
            print(min, end='')
            if minutes == 20:
                print('\n')
            for wait in range(3):
                time.sleep(15)
                print('.', end='')
        
        print(f'\nScrap No: {counter + 1}\n')
        scrap_amazon()
        counter = counter + 1

#Wait this time

In [17]:
#time.sleep(60*30)
#scrap_amazon()

#Scrap n times in t time or just one time

In [20]:
scrap_amazon()

      Log Date    | Country |        category       | status | Loaded |
 2020-08-17 13:18 |    mx   |        grocery        |   200  |   Yes  |
 2020-08-17 13:19 |    mx   |       automotive      |   200  |   Yes  |
 2020-08-17 13:19 |    mx   |          baby         |   200  |   Yes  |
 2020-08-17 13:19 |    mx   |         sports        |   200  |   Yes  |
 2020-08-17 13:19 |    mx   |     amazon-devices    |   200  |   Yes  |
 2020-08-17 13:19 |    mx   |      electronics      |   200  |   Yes  |
 2020-08-17 13:19 |    mx   |         tools         |   200  |   Yes  |
 2020-08-17 13:19 |    mx   |        kitchen        |   200  |   Yes  |
 2020-08-17 13:19 |    mx   |       industrial      |   200  |   Yes  |
 2020-08-17 13:19 |    mx   |  musical-instruments  |   200  |   Yes  |
 2020-08-17 13:20 |    mx   |          toys         |   200  |   Yes  |
 2020-08-17 13:20 |    mx   |         books         |   200  |   Yes  |
 2020-08-17 13:20 |    mx   |         music         |   200  |  

In [ ]:
case = 'steps'
#case = 'normal'

if case == 'steps':
    scrap_amazon_times(n_times= 12, minutes= 60)
else:
    scrap_amazon()

# Nueva sección